In [4]:
import pandas as pd
import numpy as np
from pandas import DataFrame

In [5]:
# Read dataframes
df_market_cap = pd.read_pickle("UsableData/MarketCap.pkl")
df_employees = pd.read_pickle("UsableData/Employees.pkl")
df_book_value = pd.read_pickle("UsableData/BookValue.pkl")
df_equal = pd.read_pickle("UsableData/Equal.pkl")
df_random = pd.read_pickle("UsableData/Random.pkl")
df_variance = pd.read_pickle("UsableData/Variance.pkl")
df_returns = pd.read_pickle("UsableData/Returns.pkl").T
df_eligible = pd.read_pickle("UsableData/EligibleStocks.pkl")

In [6]:
# Map strategy names to dataframes and if the strategy is reversed (bigger value = smaller weight)
# "Strategy_Name"[Dataframe, Bigger_is_Better] - Example bigger market cap = bigger weight -> True, bigger variance = smaller weight -> False
strategies = {"Market Cap": [df_market_cap, True], "Employees": [df_employees, True], "Book Value": [df_book_value, True], "Equal": [df_equal, True], "Random": [df_random, True], "Variance": [df_variance, False]}

In [7]:
# # Test Data to validate simulation
# from pandas import DataFrame

# i = ["A", "B", "C"]
# data = {2000: [True, True, True], 2001: [True, True, True], 2002: [True, True, True], 2003: [True, True, True]}
# df_eligible = DataFrame(data=data, index = i)
# times = pd.date_range('2000-01-31', periods=48, freq='M')
# df_returns = DataFrame(data = 1, index=times, columns=i)
# data = {1999: [1, 1, 1], 2000: [1, 1, 1], 2001: [1, 1, 1], 2002: [1, 1, 1]}
# df = DataFrame(data=data, index = i)

In [8]:
# Simulate strategies
years = df_eligible.columns.to_list()
years.insert(0, years[0]-1)
df_results = DataFrame(index=df_returns[(df_returns.index >= f"{years[0]}-12-01") & (df_returns.index <= f"{years[-1]}-12-31")].index, columns=strategies.keys())

for strategy in strategies.keys(): # Go through all strategies
    df = strategies[strategy][0]
    cash = 1
    df_results.at[f"{years[0]}-12-31", strategy] = cash # Start with 1 € by the end of year 0
    for year in years[1:]:
        cash_up_to_month = 0
        # Calculate weights
        eligible_stocks = df_eligible[df_eligible[year] == True][year].index.values
        if strategies[strategy][1]: # Default
            weight = df.loc[eligible_stocks, year-1]
        else: # Reverse weights for small variances
            weight = 1/df.loc[eligible_stocks, year-1]
        weight = weight/weight.sum()
        # Go through month and multiply returns
        returns = df_returns[(df_returns.index >= f"{year}-01-01") & (df_returns.index <= f"{year}-12-31")]
        for month in returns.index:
            total_cash_gained_up_to_month = 0
            returns_up_to_month = returns[(returns.index >= f"{year}-01-01") & (returns.index <= month)]
            for ric in eligible_stocks: # Go through every stock
                stock_return_up_to_month = returns_up_to_month[ric].values
                return_up_to_month = (np.prod(stock_return_up_to_month/100 + 1)-1)*100
                monetary_change = cash*weight[ric]*return_up_to_month/100
                total_cash_gained_up_to_month += monetary_change
            cash_up_to_month = total_cash_gained_up_to_month + cash
            df_results.at[month, strategy] = cash_up_to_month
        cash = cash_up_to_month # Add gained cash by the end of the year to total amount
df_results.to_pickle("UsableData/Results.pkl")